# MNIST_classification

A PyTorch implementation of the MNIST digit classification task using a neural network. The model is trained to recognise handwritten digits from the MNIST dataset, leveraging PyTorch's built-in functions for data loading, model building, and training.

# Section 0: Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Check if CUDA is available (for GPU acceleration)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use "cuda" if available, otherwise fallback to "cpu"
print('Device:', device)

# Section 1: Load Data

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])         # Define any data transformations (e.g., ToTensor, Normalize, etc.)
train_dataset = datasets.MNIST(root = '/kaggle/working/', train = True, download=True, transform=transform)     # Load the training dataset with the defined transform
test_dataset = datasets.MNIST(root = '/kaggle/working/', train = False, download=True, transform=transform)      # Load the test dataset with the defined transform

In [ ]:
print(train_dataset.data.shape)                   # Print shape of training data
print(test_dataset.data.shape)                    # Print shape of test data
print(type(train_dataset.data[0,0,0].item()))     # Print data type of a pixel
print(train_dataset.data.min().item(), train_dataset.data.max().item())  # Print min and max pixel values
print(train_dataset.targets.shape)                # Print shape of labels
print(train_dataset.targets[0].item())            # Print the first label

# Display the first training image
plt.imshow(train_dataset.data[0], cmap='gray')
plt.title(f'Label: {train_dataset.targets[0].item()}')
plt.show()

# Section 2: Preprocessing

In [ ]:
train_dataset.data = train_dataset.data.float()/255.0  # Normalize training data by dividing by 255.0
test_dataset.data = test_dataset.data.float()/255.0   # Normalize test data by dividing by 255.0

print(train_dataset.data.min().item(), train_dataset.data.max().item())  # Print min and max pixel values
print(train_dataset.targets[0].item())  # Print the first label

# Display the first image in the training dataset
plt.imshow(train_dataset.data[0], cmap='gray')
plt.title(f'Label: {train_dataset.targets[0].item()}')
plt.show()

# Section 3: Define Hyperparameters

In [ ]:
batch_size = 32
epochs = 25
learning_rate = 0.001

# Section 4: Define the Network

In [ ]:
class MNISTModel(nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.flatten = nn.Flatten() # Flatten the input image (e.g., from 28x28 to 784)
        self.fc1 = nn.Linear(28*28, 100)      # First fully connected layer (input: 784, output: 100)
        self.fc2 = nn.Linear(100, 200)      # Second fully connected layer (input: 100, output: 200)
        self.fc3 = nn.Linear(200, 10)      # Output layer (input: 200, output: 10 for classification)

    def forward(self, x):
        x = self.flatten(x)  # Flatten the input
        x = torch.sigmoid(self.fc1(x))  # Apply sigmoid to output of fc1
        x = torch.sigmoid(self.fc2(x))  # Apply sigmoid to output of fc2
        x = self.fc3(x) # Pass through final layer without activation
        return x

model = MNISTModel()
model = model.to("cuda") # Instantiate the model and move it to the selected device
print(model)

criterion = nn.CrossEntropyLoss()  # Define loss function (e.g., CrossEntropyLoss)
optimizer = optim.Adam(model.parameters(), lr = learning_rate)  # Define optimizer (e.g., Adam with model parameters and learning rate)


# Section 5: Train the Network

In [ ]:
# Section 5: Train the Network
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  # Create DataLoader for training data with batch_size and shuffle=True
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)   # Create DataLoader for test data with batch_size and shuffle=False

for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to("cuda"), labels.to("cuda")
        # Move images and labels to the device (CPU or GPU)
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass through the model
        outputs = model(images)
        
        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        
        # Optimizer step
        optimizer.step()
        
        # Accumulate loss into running_loss
        running_loss += loss.item()
   
    # Print average loss for the epoch
    print(f'Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}')

# Section 6: Perform Inference

In [ ]:
# Section 6: Perform Inference
model.eval()  # Set the model to evaluation mode
predictions = []

with torch.no_grad():
    for images, _ in test_loader:
        images = images.to("cuda") # Move images to the device
        # Perform a forward pass through the model
        outputs = model(images)
        
        # Get the predicted class labels (e.g., using torch.max)
        _, predicted = torch.max(outputs, 1)
        
        # Append predictions to the predictions list
        predictions.extend(predicted.cpu().numpy())

# Section 7: Output

In [ ]:
index = 0  # Choose an index to visualize

# Print the predicted digit at the chosen index
print(f'Predicted digit: {predictions[index]}')

# Display the corresponding image from the test dataset
plt.imshow(test_dataset.data[index], cmap='gray')
plt.title(f'Predicted: {predictions[index]}')
plt.show()